# 🏗️ Module 3: Алокатори пам’яті (The Deep Dive)

**Мета:** Зрозуміти, як Python взаємодіє з RAM та OS. Найважливіший розділ для High Load.

## 1. Ієрархія алокаторів пам’яті

Python має **трирівневу** систему алокації пам’яті:

```
+----------------------------------------------+
|  Layer 2: Pymalloc (Object Allocator)        |  ← Об’єкти ≤ 512 байт
|  Арени → Пули → Блоки                      |
+----------------------------------------------+
|  Layer 1: Raw Domain (PyMem_Malloc)          |  ← Внутрішні буфери CPython
+----------------------------------------------+
|  Layer 0: OS Allocator (malloc / mmap)       |  ← Об’єкти > 512 байт
+----------------------------------------------+
```

Коли ви пишете `x = 42`, Python **не** викликає `malloc()` для кожного об’єкта. Це було б катастрофічно повільно. Замість цього він використовує **Pymalloc** — власний менеджер, оптимізований для маленьких об’єктів.

## 2. Архітектура Pymalloc: Arenas → Pools → Blocks

### Arena (256 KB)
Найбільша одиниця. Python запитує у OS блок пам’яті 256 KB через `mmap()` (або `VirtualAlloc` на Windows).

### Pool (4 KB)
Кожна арена містить 64 пули (256 KB / 4 KB = 64). Кожен пул обслуговує блоки **одного розміру**.

### Block (8–512 байт)
Фіксовані комірки пам’яті. Розмір блоку кратний 8: 8, 16, 24, 32, ..., 512 байт.

```
Arena (256 KB)
├── Pool 0 (4 KB) ─ блоки по 16 bytes
├── Pool 1 (4 KB) ─ блоки по 32 bytes
├── Pool 2 (4 KB) ─ блоки по 16 bytes
├── ...
└── Pool 63 (4 KB) ─ блоки по 64 bytes
```

## 3. Практика: Вимірювання розмірів об’єктів

Подивімось, як об’єкти різних типів потрапляють у різні алокатори.

In [1]:
import sys

objects = {
    'int(0)':       0,
    'int(1000)':    1000,
    'float(3.14)':  3.14,
    'str("")':      "",
    'str("hello")': "hello",
    'str("a"*600)': "a" * 600,
    'list([])':     [],
    'list(range(10))': list(range(10)),
    'dict({})':     {},
    'tuple(())':    (),
    'set()':        set(),
    'object()':     object(),
}

print(f"{'Object':<20} {'Size (bytes)':>12}  {'Allocator'}")
print("-" * 55)
for name, obj in objects.items():
    size = sys.getsizeof(obj)
    allocator = "Pymalloc" if size <= 512 else "OS malloc"
    print(f"{name:<20} {size:>12}  {allocator}")

Object               Size (bytes)  Allocator
-------------------------------------------------------
int(0)                         28  Pymalloc
int(1000)                      28  Pymalloc
float(3.14)                    24  Pymalloc
str("")                        41  Pymalloc
str("hello")                   46  Pymalloc
str("a"*600)                  641  OS malloc
list([])                       56  Pymalloc
list(range(10))               136  Pymalloc
dict({})                       64  Pymalloc
tuple(())                      48  Pymalloc
set()                         216  Pymalloc
object()                       16  Pymalloc


## 4. Практика: Показуємо роботу Pymalloc через `_tracemalloc`

Ми можемо побачити, скільки пам’яті Python виділив через свій алокатор.

In [2]:
import tracemalloc

tracemalloc.start()

# Створюємо багато маленьких об'єктів (потрапляють у Pymalloc)
small_objects = [object() for _ in range(10000)]

# Створюємо великі об'єкти (підуть через OS malloc)
# Використовуємо bytearray — кожен об'єкт унікальний (на відміну від рядків, які можуть інтернуватись)
big_objects = [bytearray(1000) for _ in range(1000)]

snapshot = tracemalloc.take_snapshot()
stats = snapshot.statistics('lineno')

print("Top 5 memory allocations:")
for stat in stats[:5]:
    print(f"  {stat}")

current, peak = tracemalloc.get_traced_memory()
print(f"\nCurrent memory: {current / 1024:.1f} KB")
print(f"Peak memory:    {peak / 1024:.1f} KB")

tracemalloc.stop()

Top 5 memory allocations:
  C:\Users\mcipa\AppData\Local\Temp\ipykernel_5512\2711580416.py:10: size=1041 KiB, count=2001, average=533 B
  C:\Users\mcipa\AppData\Local\Temp\ipykernel_5512\2711580416.py:6: size=239 KiB, count=10001, average=25 B
  C:\Users\mcipa\AppData\Local\Programs\Python\Python314\Lib\codeop.py:117: size=574 B, count=6, average=96 B
  D:\myprojects\python_memory\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3641: size=304 B, count=1, average=304 B
  D:\myprojects\python_memory\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3701: size=168 B, count=1, average=168 B

Current memory: 1287.0 KB
Peak memory:    1307.2 KB


## 5. Fragmentation: Чому пам’ять не повертається в OS

Це ключова проблема для **long-running** процесів (web-сервери, workerи).

### Проблема:
- Арена (256 KB) повертається в OS **тільки коли ВСІ її блоки вільні**
- Якщо хоча б **один** об’єкт в арені живий — вся арена залишається в пам’яті
- Це створює **«дірки» (holes)** — пам’ять вільна для Python, але не повернута OS

```
Arena (256 KB):
[███░░░█░░░░░░█░░]  ← 3 лише живих об'єкти, але вся арена занята!
```

In [3]:
import os
import gc

try:
    import psutil
    def get_rss():
        return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
except ImportError:
    import resource
    def get_rss():
        return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

baseline = get_rss()
print(f"Baseline RSS: {baseline:.1f} MB")

# Створюємо 500К маленьких об'єктів
objects = [bytearray(50) for _ in range(500_000)]
after_alloc = get_rss()
print(f"After allocation: {after_alloc:.1f} MB (+{after_alloc - baseline:.1f} MB)")

# Видаляємо 90% об'єктів (але кожний 10-й лишаємо — створюємо фрагментацію)
for i in range(len(objects)):
    if i % 10 != 0:  # залишаємо кожний 10-й
        objects[i] = None

gc.collect()
after_fragmented = get_rss()
freed_mb = after_alloc - after_fragmented
print(f"After deleting 90% (fragmented): {after_fragmented:.1f} MB")
print(f"  → Звільнили 90% об'єктів, але RSS впав лише на {abs(freed_mb):.1f} MB")

# Порівняння: видаляємо ВСЕ
objects = None
gc.collect()
after_all = get_rss()
print(f"After deleting ALL: {after_all:.1f} MB")

print(f"\nФрагментація: після видалення 90% RSS = {after_fragmented:.1f} MB")
print(f"Лише після видалення 100% RSS знизився до {after_all:.1f} MB")
print("Кожний 10-й об'єкт 'тримав' цілу арену в пам'яті!")

Baseline RSS: 82.6 MB
After allocation: 149.2 MB (+66.6 MB)
After deleting 90% (fragmented): 149.2 MB
  → Звільнили 90% об'єктів, але RSS впав лише на 0.0 MB
After deleting ALL: 87.4 MB

Фрагментація: після видалення 90% RSS = 149.2 MB
Лише після видалення 100% RSS знизився до 87.4 MB
Кожний 10-й об'єкт 'тримав' цілу арену в пам'яті!


## 6. Memory Footprint: Як виміряти реальне споживання

Існують два ключових показники:

| Метрика | Що вимірює | Коли використовувати |
|--------|-------------|--------------------|
| **RSS** (Resident Set Size) | Скільки фізичної RAM займає процес | Моніторинг в production |
| **VSZ** (Virtual Memory Size) | Зарезервована віртуальна пам’ять | Рідко корисно |

In [4]:
import os
import sys

try:
    import psutil
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    print(f"RSS (Physical):  {mem_info.rss / 1024 / 1024:.2f} MB")
    print(f"VMS (Virtual):   {mem_info.vms / 1024 / 1024:.2f} MB")
except ImportError:
    print("psutil not installed. Install: pip install psutil")

# sys.getsizeof — розмір SHALLOW (без вкладених об'єктів)
data = [[1, 2, 3] for _ in range(100)]
shallow = sys.getsizeof(data)
deep = sys.getsizeof(data) + sum(sys.getsizeof(item) for item in data)

print(f"\nShallow size of list[100]: {shallow} bytes")
print(f"Deep size (1 рівень):       {deep} bytes")
print("\n❗ sys.getsizeof() показує лише 'shallow' розмір!")
print("Навіть ручний 'deep size' неточний — shared objects (кешовані числа)")
print("рахуються багаторазово. Для точного підрахунку → pympler.asizeof (Module 8).")

RSS (Physical):  86.39 MB
VMS (Virtual):   88.48 MB

Shallow size of list[100]: 920 bytes
Deep size (1 рівень):       9720 bytes

❗ sys.getsizeof() показує лише 'shallow' розмір!
Навіть ручний 'deep size' неточний — shared objects (кешовані числа)
рахуються багаторазово. Для точного підрахунку → pympler.asizeof (Module 8).


## 7. Практичні висновки

1. **Об’єкти ≤ 512 байт** → Pymalloc (швидко, але створює фрагментацію)
2. **Об’єкти > 512 байт** → OS malloc (повільніше, але пам’ять повертається OS)
3. **Long-running процеси** — уникайте створення/видалення мільйонів маленьких об’єктів
4. **Фрагментація** — використовуйте `array`, `numpy`, або pre-allocated буфери
5. **Моніторинг** — завжди дивіться на RSS, а не на `sys.getsizeof()`